In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("../../")
sys.path.append("../../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import SuperNet, AESuperNet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [4]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')

In [5]:
net1 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  98%|█████████▊| 89/91 [00:13<00:00,  9.38batch/s]
                                                                    INFO: Validation Dice Loss: 0.08108492842414877
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 23.31batch/s]
                                                                    INFO: Validation Dice Loss: 0.08589861992296281
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 27.56batch/s]
                                                                    INFO: Validation Dice Loss: 0.08703502398598326
Validation round:  98%|█████████▊| 89

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.55batch/s]
                                                                    INFO: Validation Dice Loss: 0.07933948180832706
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.39batch/s]
                                                                    INFO: Validation Dice Loss: 0.07931823078747634
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.24batch/s]
                                                                    INFO: Validation Dice Loss: 0.07955108435599359
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.00batch/s]
                                                                    INFO: Validation Dice Loss: 0.07916119015151328
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.67batch/s]
                                                                    INFO: Validation Dice Loss: 0.07879141832773502
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.49batch/s]
      

                                                                    INFO: Validation Dice Loss: 0.0634023632322039
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.95batch/s]
                                                                    INFO: Validation Dice Loss: 0.062671439742649
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.52batch/s]
                                                                    INFO: Validation Dice Loss: 0.062323672021483326
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.58batch/s]
                                                                    INFO: Validation Dice Loss: 0.06223380066208787
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.06154001626994584
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.74batch/s]
                                                                    INFO: Val

Validation round:  97%|█████████▋| 88/91 [00:05<00:00, 21.75batch/s]
                                                                    INFO: Validation Dice Loss: 0.05849173337548644
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.70batch/s]
                                                                    INFO: Validation Dice Loss: 0.05799293100506395
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.68batch/s]
                                                                    INFO: Validation Dice Loss: 0.05883786499827773
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.97batch/s]
                                                                    INFO: Validation Dice Loss: 0.05694598508077663
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.056778735921278106
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.49batch/s]
     

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.17batch/s]
                                                                    INFO: Validation Dice Loss: 0.05087442368596465
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.05076440497414096
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.05094263989191789
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.40batch/s]
                                                                    INFO: Validation Dice Loss: 0.05122872311007846
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.14batch/s]
                                                                    INFO: Validation Dice Loss: 0.050628769856232866
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 16.58batch/s]
     

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.14batch/s]
                                                                    INFO: Validation Dice Loss: 0.05040809307452086
Epoch 5/8: 100%|██████████| 5865/5865 [01:29<00:00, 65.70img/s, loss (batch)=0.117]  
INFO: Checkpoint 5 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.24batch/s]
                                                                    INFO: Validation Dice Loss: 0.050369139801669904
Validation round:  98%|█████████▊| 89/91 [00:06<00:00, 14.60batch/s]
                                                                    INFO: Validation Dice Loss: 0.05044014137852323
Validation round:  99%|█████████▉| 90/91 [00:06<00:00, 15.02batch/s]
                                                                    INFO: Validation Dice Loss: 0.05079484084150293
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.31batch/s]
                                                                    INFO: Val

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.23batch/s]
                                                                    INFO: Validation Dice Loss: 0.049312993147216
Epoch 6/8: 100%|██████████| 5865/5865 [01:31<00:00, 64.42img/s, loss (batch)=0.281]  
INFO: Checkpoint 6 saved !
Validation round:  99%|█████████▉| 90/91 [00:06<00:00, 14.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.049461924112760104
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 14.32batch/s]
                                                                    INFO: Validation Dice Loss: 0.04940126426927336
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 13.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.04924699000932358
Validation round: 100%|██████████| 91/91 [00:06<00:00, 14.54batch/s]
                                                                    INFO: Valid

Validation round:  99%|█████████▉| 90/91 [00:13<00:00,  3.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.04935300939685696
Validation round: 100%|██████████| 91/91 [00:13<00:00,  4.93batch/s]
                                                                    INFO: Validation Dice Loss: 0.04893846163055399
Validation round:  98%|█████████▊| 89/91 [00:12<00:00,  6.55batch/s]
                                                                    INFO: Validation Dice Loss: 0.049429547328215376
Epoch 7/8: 100%|██████████| 5865/5865 [02:42<00:00, 36.00img/s, loss (batch)=0.112] 
INFO: Checkpoint 7 saved !
Validation round: 100%|██████████| 91/91 [00:11<00:00,  3.67batch/s]
                                                                    INFO: Validation Dice Loss: 0.04953443774810204
Validation round:  99%|█████████▉| 90/91 [00:10<00:00,  6.52batch/s]
                                                                    INFO: Vali

Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 14.60batch/s]
                                                                    INFO: Validation Dice Loss: 0.04857030338965929
Validation round: 100%|██████████| 91/91 [00:07<00:00, 14.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.048814745067240116
Validation round:  97%|█████████▋| 88/91 [00:06<00:00, 13.31batch/s]
                                                                    INFO: Validation Dice Loss: 0.04861482093622396
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 10.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.04863029562837475
Epoch 8/8: 100%|██████████| 5865/5865 [02:26<00:00, 39.92img/s, loss (batch)=0.0726] 
INFO: Checkpoint 8 saved !


In [6]:
net2 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=128, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 14.05batch/s]
                                                                    INFO: Validation Dice Loss: 0.12204440118192317
Validation round: 100%|██████████| 91/91 [00:07<00:00, 14.88batch/s]
                                                                    INFO: Validation Dice Loss: 0.1495231779722067
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 13.77batch/s]
                                                                    INFO: Validation Dice Loss: 0.08307790314103221
Validation round:  99%|█████████▉| 90/

Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 13.03batch/s]
                                                                    INFO: Validation Dice Loss: 0.08065567427611613
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.08245928258031279
Epoch 1/8: 100%|██████████| 5865/5865 [01:52<00:00, 52.18img/s, loss (batch)=0.445] 
INFO: Checkpoint 1 saved !
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 15.26batch/s]
                                                                    INFO: Validation Dice Loss: 0.07590559116759144
Validation round: 100%|██████████| 91/91 [00:07<00:00, 13.58batch/s]
                                                                    INFO: Validation Dice Loss: 0.08091009793045757
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.16batch/s]
                                                                    INFO: Valid

Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 12.99batch/s]
                                                                    INFO: Validation Dice Loss: 0.07543931533019621
Validation round:  97%|█████████▋| 88/91 [00:07<00:00, 16.03batch/s]
                                                                    INFO: Validation Dice Loss: 0.07083881810143754
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 14.02batch/s]
                                                                    INFO: Validation Dice Loss: 0.07278313913515636
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.95batch/s]
                                                                    INFO: Validation Dice Loss: 0.07171290924588403
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 13.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.07245101239327546
Epoch 2/8: 100%|██████████| 5865/5865 [02:00<00:00, 48.70img/s, loss (batch

Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 13.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.07000893297103736
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 13.25batch/s]
                                                                    INFO: Validation Dice Loss: 0.06957944654501401
Validation round: 100%|██████████| 91/91 [00:07<00:00, 13.48batch/s]
                                                                    INFO: Validation Dice Loss: 0.0702854539011861
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.17batch/s]
                                                                    INFO: Validation Dice Loss: 0.06820937655456774
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.90batch/s]
                                                                    INFO: Validation Dice Loss: 0.06803884480025742
Validation round: 100%|██████████| 91/91 [00:07<00:00, 11.73batch/s]
       

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.06742226700861376
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.97batch/s]
                                                                    INFO: Validation Dice Loss: 0.06649888896352642
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.06665925910839668
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.62batch/s]
                                                                    INFO: Validation Dice Loss: 0.0665105429815722
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.06799421238375235
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.32batch/s]
       

...

Validation round: 100%|██████████| 91/91 [00:07<00:00, 15.03batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705081331860888
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 13.89batch/s]
                                                                    INFO: Validation Dice Loss: 0.0670498686504888
Epoch 5/8: 100%|██████████| 5865/5865 [01:48<00:00, 54.17img/s, loss (batch)=0.282] 
INFO: Checkpoint 5 saved !
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 13.07batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705024723823254
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.77batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705006326620395
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.68batch/s]
                                                                    INFO: Valida

Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 13.70batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705123914794607
Validation round: 100%|██████████| 91/91 [00:07<00:00, 11.71batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705104068412886
Validation round: 100%|██████████| 91/91 [00:07<00:00, 15.06batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705042204031578
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 14.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.0670521650668029
Epoch 6/8: 100%|██████████| 5865/5865 [02:01<00:00, 48.17img/s, loss (batch)=0.256] 
INFO: Checkpoint 6 saved !
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 13.83batch/s]
                                                                    INFO: Valida

Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 12.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705261300225834
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 14.69batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705321633553767
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 12.74batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705313356040599
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 14.88batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705394305370665
Validation round: 100%|██████████| 91/91 [00:07<00:00, 13.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705190716208992
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.27batch/s]
      

Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 11.73batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705350559818876
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 13.71batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705402746632859
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705364216487486
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 11.15batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705267940248762
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.80batch/s]
                                                                    INFO: Validation Dice Loss: 0.0670540955040481
Validation round:  99%|█████████▉| 90/91 [00:07<00:00, 12.99batch/s]
       

                                                                    INFO: Validation Dice Loss: 0.06705392602380815
Validation round: 100%|██████████| 91/91 [00:07<00:00, 12.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705501323545372
Validation round:  98%|█████████▊| 89/91 [00:07<00:00, 12.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.06705395869173847
Epoch 8/8: 100%|██████████| 5865/5865 [02:00<00:00, 48.51img/s, loss (batch)=0.258] 
INFO: Checkpoint 8 saved !


In [7]:
net3 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=200, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net3.to(device=device)
train_scores3, val_scores3, train_var_3, val_var_3 = train_net(net=net3,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round: 100%|██████████| 91/91 [00:07<00:00, 14.02batch/s]
                                                                    INFO: Validation Dice Loss: 0.3635219250406538
Validation round:  99%|█████████▉| 90/91 [00:06<00:00, 16.78batch/s]
                                                                    INFO: Validation Dice Loss: 0.3832572027222141
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.29094966525559895
Validation round:  99%|█████████▉| 90/9

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.3540599133287157
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.38batch/s]
                                                                    INFO: Validation Dice Loss: 0.34015773023877827
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.82batch/s]
                                                                    INFO: Validation Dice Loss: 0.3446269428337013
Epoch 1/8: 100%|██████████| 5865/5865 [02:14<00:00, 43.57img/s, loss (batch)=0.623]
INFO: Checkpoint 1 saved !
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.34405955115517417
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.30batch/s]
                                                                    INFO: Validati

Validation round: 100%|██████████| 91/91 [00:05<00:00, 15.94batch/s]
                                                                    INFO: Validation Dice Loss: 0.3499213267158676
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.34938927993669616
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.3508830388168712
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.3499284825482211
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.33batch/s]
                                                                    INFO: Validation Dice Loss: 0.3497471203515818
Epoch 2/8: 100%|██████████| 5865/5865 [02:10<00:00, 44.85img/s, loss (batch)=0.

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.34971061217915883
Validation round: 100%|██████████| 91/91 [00:06<00:00, 17.16batch/s]
                                                                    INFO: Validation Dice Loss: 0.34971726955948296
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.34967758989596104
Validation round: 100%|██████████| 91/91 [00:05<00:00, 16.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.349670776954064
Validation round: 100%|██████████| 91/91 [00:06<00:00, 14.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496893074486282
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.40batch/s]
         

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.34971413835064397
Epoch 3/8: 100%|██████████| 5865/5865 [02:12<00:00, 44.36img/s, loss (batch)=0.621]
INFO: Checkpoint 3 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496658536759052
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 15.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496578355411907
Validation round: 100%|██████████| 91/91 [00:05<00:00, 17.04batch/s]
                                                                    INFO: Validation Dice Loss: 0.34966959010113724
Validation round:  98%|█████████▊| 89/91 [00:06<00:00, 15.07batch/s]
                                                                    INFO: Validati

...

                                                                    INFO: Validation Dice Loss: 0.3496427467236152
Epoch 5/8: 100%|██████████| 5865/5865 [02:12<00:00, 44.30img/s, loss (batch)=0.74] 
INFO: Checkpoint 5 saved !
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.34957556344650603
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.15batch/s]
                                                                    INFO: Validation Dice Loss: 0.34956419304177
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 26.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495633133820125
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.24batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495691867320092
Validation round: 100%|██████████| 91/9

Validation round: 100%|██████████| 91/91 [00:05<00:00, 17.31batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496449560254485
Validation round: 100%|██████████| 91/91 [00:05<00:00, 17.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496298704828535
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.3496136023448064
Epoch 6/8: 100%|██████████| 5865/5865 [02:14<00:00, 43.66img/s, loss (batch)=0.62] 
INFO: Checkpoint 6 saved !
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.75batch/s]
                                                                    INFO: Validation Dice Loss: 0.34960357468206804
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 20.95batch/s]
                                                                    INFO: Validatio

Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495730764263279
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.98batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495384130504105
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495638200214931
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.34956907767515916
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.34958169355497254
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.84batch/s]
         

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.349583679830635
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.10batch/s]
                                                                    INFO: Validation Dice Loss: 0.34960235966430914
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.62batch/s]
                                                                    INFO: Validation Dice Loss: 0.34957669986473333
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.56batch/s]
                                                                    INFO: Validation Dice Loss: 0.349572370668034
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.3494985765153235
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.70batch/s]
           

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.3495727633381938
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.34956131662641254
Epoch 8/8: 100%|██████████| 5865/5865 [02:16<00:00, 43.09img/s, loss (batch)=0.624]
INFO: Checkpoint 8 saved !


In [8]:
net4 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=256, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net4.to(device=device)
train_scores4, val_scores4, train_var_4, val_var_4 = train_net(net=net4,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  97%|█████████▋| 88/91 [00:05<00:00, 18.91batch/s]
                                                                    INFO: Validation Dice Loss: 0.19016237409560235
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.18022841968379177
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.02batch/s]
                                                                    INFO: Validation Dice Loss: 0.17419522869717943
Validation round:  98%|█████████▊| 89

Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.60batch/s]
                                                                    INFO: Validation Dice Loss: 0.17587765363546518
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.54batch/s]
                                                                    INFO: Validation Dice Loss: 0.17094334354112437
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.48batch/s]
                                                                    INFO: Validation Dice Loss: 0.17555475136735937
Epoch 1/8: 100%|██████████| 5865/5865 [02:21<00:00, 41.43img/s, loss (batch)=0.624]
INFO: Checkpoint 1 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.67batch/s]
                                                                    INFO: Validation Dice Loss: 0.18047469166608957
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.00batch/s]
                                                                    INFO: Valida

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.04batch/s]
                                                                    INFO: Validation Dice Loss: 0.17248227979455674
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.00batch/s]
                                                                    INFO: Validation Dice Loss: 0.172216398866622
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.68batch/s]
                                                                    INFO: Validation Dice Loss: 0.17378106205673008
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 16.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.1735615577999052
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.17375678102393727
Validation round:  70%|███████   | 64/91 [00:04<00:01, 15.82batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 25.25batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332480667711614
Epoch 2/8: 100%|██████████| 5865/5865 [02:26<00:00, 39.94img/s, loss (batch)=0.599]
INFO: Checkpoint 2 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.32batch/s]
                                                                    INFO: Validation Dice Loss: 0.17329688828725082
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.17337539530062412
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.33batch/s]
                                                                    INFO: Validation Dice Loss: 0.17335487672915825
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.36batch/s]
                                                                    INFO: Valida

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.25batch/s]
                                                                    INFO: Validation Dice Loss: 0.1733347686764958
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.05batch/s]
                                                                    INFO: Validation Dice Loss: 0.17333567109736767
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 23.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.17333626583382322
Validation round: 100%|██████████| 91/91 [00:05<00:00, 21.02batch/s]
                                                                    INFO: Validation Dice Loss: 0.17333359885346758
Epoch 3/8: 100%|██████████| 5865/5865 [02:29<00:00, 39.21img/s, loss (batch)=0.561]
INFO: Checkpoint 3 saved !
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.05batch/s]
                                                                    INFO: Validat

...

Validation round: 100%|██████████| 91/91 [00:05<00:00, 21.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332260851021652
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 15.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332400594438827
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332370251744658
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.92batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332404115042843
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.03batch/s]
                                                                    INFO: Validation Dice Loss: 0.1733248702117375
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.14batch/s]
       

Validation round: 100%|██████████| 91/91 [00:05<00:00, 22.32batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332311678718734
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332118176496947
Epoch 6/8: 100%|██████████| 5865/5865 [02:13<00:00, 44.07img/s, loss (batch)=0.568]
INFO: Checkpoint 6 saved !
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.26batch/s]
                                                                    INFO: Validation Dice Loss: 0.1733202372605984
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.83batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332070541905834
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.44batch/s]
                                                                    INFO: Validat

Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.62batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331939280688108
Validation round: 100%|██████████| 91/91 [00:05<00:00, 21.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331852690204158
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.1733205076102372
Validation round:  98%|█████████▊| 89/91 [00:06<00:00, 16.98batch/s]
                                                                    INFO: Validation Dice Loss: 0.17332118831493043
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 16.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.1733194594527339
Epoch 7/8: 100%|██████████| 5865/5865 [02:13<00:00, 44.08img/s, loss (batch)=

Validation round: 100%|██████████| 91/91 [00:05<00:00, 17.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331785880602324
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331853197826133
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.29batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331793445807236
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 20.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331986653280781
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.05batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331957882577245
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.51batch/s]
      

Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.17331713470783863
Epoch 8/8: 100%|██████████| 5865/5865 [02:13<00:00, 44.07img/s, loss (batch)=0.562]
INFO: Checkpoint 8 saved !


In [9]:
net5 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=64, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net5.to(device=device)
train_scores5, val_scores5, train_var_5, val_var_5 = train_net(net=net5,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 16.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.20003606772029792
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.99batch/s]
                                                                    INFO: Validation Dice Loss: 0.20972144358105713
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.68batch/s]
                                                                    INFO: Validation Dice Loss: 0.19974993824303805
Validation round:  98%|█████████▊| 89

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.68batch/s]
                                                                    INFO: Validation Dice Loss: 0.1903729320882441
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.17178579853786216
Validation round: 100%|██████████| 91/91 [00:05<00:00, 16.14batch/s]
                                                                    INFO: Validation Dice Loss: 0.16616431402636098
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.79batch/s]
                                                                    INFO: Validation Dice Loss: 0.16370577356972538
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.24batch/s]
                                                                    INFO: Validation Dice Loss: 0.16507116991740006
Validation round:  96%|█████████▌| 87/91 [00:05<00:00, 17.55batch/s]
       

Epoch 2/8: 100%|██████████| 5865/5865 [01:29<00:00, 65.48img/s, loss (batch)=0.394] 
INFO: Checkpoint 2 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.11606636430535998
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.11527256872300264
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.11368435686761207
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.11227694227472766
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.74batch/s]
                                                                    INFO: Valid

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.94batch/s]
                                                                    INFO: Validation Dice Loss: 0.09952936386996573
Epoch 3/8: 100%|██████████| 5865/5865 [01:29<00:00, 65.67img/s, loss (batch)=0.278] 
INFO: Checkpoint 3 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.48batch/s]
                                                                    INFO: Validation Dice Loss: 0.10065814336905113
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 14.85batch/s]
                                                                    INFO: Validation Dice Loss: 0.10081485478760122
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.10033576108597136
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.25batch/s]
                                                                    INFO: Valid

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.95batch/s]
                                                                    INFO: Validation Dice Loss: 0.09595670248125936
Epoch 4/8: 100%|██████████| 5865/5865 [01:30<00:00, 65.16img/s, loss (batch)=0.207] 
INFO: Checkpoint 4 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.55batch/s]
                                                                    INFO: Validation Dice Loss: 0.09478339954064442
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.15batch/s]
                                                                    INFO: Validation Dice Loss: 0.09492648458415336
Validation round:  96%|█████████▌| 87/91 [00:04<00:00, 22.55batch/s]
                                                                    INFO: Validation Dice Loss: 0.09548334277920671
Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.75batch/s]
                                                                    INFO: Valid

Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 23.45batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470642227065432
Epoch 5/8: 100%|██████████| 5865/5865 [01:10<00:00, 83.47img/s, loss (batch)=0.216] 
INFO: Checkpoint 5 saved !
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.26batch/s]
                                                                    INFO: Validation Dice Loss: 0.09449005028703711
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.54batch/s]
                                                                    INFO: Validation Dice Loss: 0.09482469712639903
Validation round:  99%|█████████▉| 90/91 [00:03<00:00, 24.50batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470965131953522
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 19.54batch/s]
                                                                    INFO: Valid

Validation round: 100%|██████████| 91/91 [00:03<00:00, 29.23batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470636389412723
Validation round: 100%|██████████| 91/91 [00:04<00:00, 22.08batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470645166360415
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470586298586248
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470564536340945
Validation round:  93%|█████████▎| 85/91 [00:04<00:00, 21.69batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470471846205848
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 25.97batch/s]
      

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.0947044835640834
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 23.92batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470496907993987
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470528953678005
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 21.00batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470434945363265
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 20.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.09470446489669465
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 20.39batch/s]
       

In [10]:
net6 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=128, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net6.to(device=device)
train_scores6, val_scores6, train_var_6, val_var_6 = train_net(net=net6,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.78batch/s]
                                                                    INFO: Validation Dice Loss: 0.20322547509120062
Validation round: 100%|██████████| 91/91 [00:04<00:00, 21.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.20502695326621717
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.20batch/s]
                                                                    INFO: Validation Dice Loss: 0.19933880718199762
Validation round:  99%|█████████▉| 90

Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 26.89batch/s]
                                                                    INFO: Validation Dice Loss: 0.18281522278602308
Validation round:  95%|█████████▍| 86/91 [00:04<00:00, 21.63batch/s]
                                                                    INFO: Validation Dice Loss: 0.1856825867524514
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.61batch/s]
                                                                    INFO: Validation Dice Loss: 0.1870394332723303
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.19033543929293917
Validation round: 100%|██████████| 91/91 [00:04<00:00, 23.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.18095480789850046
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.33batch/s]
        

Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 23.21batch/s]
                                                                    INFO: Validation Dice Loss: 0.18163773450222645
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.23batch/s]
                                                                    INFO: Validation Dice Loss: 0.18146773141164046
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 25.40batch/s]
                                                                    INFO: Validation Dice Loss: 0.18166733999828716
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.31batch/s]
                                                                    INFO: Validation Dice Loss: 0.1816986674790854
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 24.90batch/s]
                                                                    INFO: Validation Dice Loss: 0.18171211242020785
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.20batch/s]
       

                                                                    INFO: Validation Dice Loss: 0.18172568213808668
Epoch 3/8: 100%|██████████| 5865/5865 [01:25<00:00, 68.97img/s, loss (batch)=0.595] 
INFO: Checkpoint 3 saved !
Validation round:  98%|█████████▊| 89/91 [00:03<00:00, 26.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172700703144073
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172753561329055
Validation round:  95%|█████████▍| 86/91 [00:04<00:00, 20.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172618501133972
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.95batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172619974875187
Validation round:  97%|█████████

Validation round:  97%|█████████▋| 88/91 [00:05<00:00, 26.90batch/s]
                                                                    INFO: Validation Dice Loss: 0.1817278259403103
Epoch 4/8: 100%|██████████| 5865/5865 [01:26<00:00, 67.49img/s, loss (batch)=0.593] 
INFO: Checkpoint 4 saved !
Validation round: 100%|██████████| 91/91 [00:04<00:00, 23.80batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172821615423476
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.61batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172486339296615
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 21.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.1817251735336178
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.09batch/s]
                                                                    INFO: Validat

Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 23.74batch/s]
                                                                    INFO: Validation Dice Loss: 0.1817270258625785
Epoch 5/8: 100%|██████████| 5865/5865 [01:26<00:00, 68.10img/s, loss (batch)=0.604] 
INFO: Checkpoint 5 saved !
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 24.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172758359175462
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172669820078127
Validation round:  96%|█████████▌| 87/91 [00:04<00:00, 20.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172708825095668
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.62batch/s]
                                                                    INFO: Valida

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.66batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172231955187662
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 26.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172277755789704
Epoch 6/8: 100%|██████████| 5865/5865 [01:23<00:00, 69.93img/s, loss (batch)=0.593] 
INFO: Checkpoint 6 saved !
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 22.22batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172245448107247
Validation round: 100%|██████████| 91/91 [00:04<00:00, 20.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172321247530507
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.08batch/s]
                                                                    INFO: Valid

Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 23.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.1817266923058164
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.14batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172663957863064
Epoch 7/8: 100%|██████████| 5865/5865 [01:24<00:00, 69.25img/s, loss (batch)=0.595] 
INFO: Checkpoint 7 saved !
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.31batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172884642422854
Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.93batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172776649941455
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 19.50batch/s]
                                                                    INFO: Valida

Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 22.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172836827707814
Validation round:  95%|█████████▍| 86/91 [00:04<00:00, 21.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.18172862798303038
Epoch 8/8: 100%|██████████| 5865/5865 [01:24<00:00, 69.72img/s, loss (batch)=0.599] 
INFO: Checkpoint 8 saved !


In [11]:
net7 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=200, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net7.to(device=device)
train_scores7, val_scores7, train_var_7, val_var_7 = train_net(net=net7,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.91batch/s]
                                                                    INFO: Validation Dice Loss: 0.9050013508115496
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.9077422147268778
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.00batch/s]
                                                                    INFO: Validation Dice Loss: 0.9064704618611179
Validation round: 100%|██████████| 91/91

Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 26.08batch/s]
                                                                    INFO: Validation Dice Loss: 0.34488878472820744
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.54batch/s]
                                                                    INFO: Validation Dice Loss: 0.36610298988583323
Epoch 1/8: 100%|██████████| 5865/5865 [02:07<00:00, 46.00img/s, loss (batch)=0.709]
INFO: Checkpoint 1 saved !
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.01batch/s]
                                                                    INFO: Validation Dice Loss: 0.4653843674030933
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.93batch/s]
                                                                    INFO: Validation Dice Loss: 0.3704955348601708
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.95batch/s]
                                                                    INFO: Validati

Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.71batch/s]
                                                                    INFO: Validation Dice Loss: 0.3695466479102334
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.36740534744419895
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.99batch/s]
                                                                    INFO: Validation Dice Loss: 0.37258881133991284
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 23.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.3661550764854138
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 26.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.3699733772120633
Epoch 2/8: 100%|██████████| 5865/5865 [02:11<00:00, 44.55img/s, loss (batch)=0

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.19batch/s]
                                                                    INFO: Validation Dice Loss: 0.37177033935274395
Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.79batch/s]
                                                                    INFO: Validation Dice Loss: 0.3716061871785384
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.37149656932432573
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.37148201301857664
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.3714945054316259
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.25batch/s]
        

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 23.36batch/s]
                                                                    INFO: Validation Dice Loss: 0.37148296865788133
Epoch 3/8: 100%|██████████| 5865/5865 [02:11<00:00, 44.74img/s, loss (batch)=0.696]
INFO: Checkpoint 3 saved !
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.91batch/s]
                                                                    INFO: Validation Dice Loss: 0.37144407859215367
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.61batch/s]
                                                                    INFO: Validation Dice Loss: 0.37144040504654685
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.63batch/s]
                                                                    INFO: Validation Dice Loss: 0.37142510368273807
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 25.10batch/s]
                                                                    INFO: Valida

...

Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.70batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713925490667532
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713677433820871
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713587250683334
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713629599455949
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.33batch/s]
                                                                    INFO: Validation Dice Loss: 0.37137140579276035
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.63batch/s]
          

Validation round: 100%|██████████| 91/91 [00:04<00:00, 23.54batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713684684627659
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.71batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713690180044908
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 23.24batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713731647847773
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.32batch/s]
                                                                    INFO: Validation Dice Loss: 0.37139157476005974
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.07batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713690622167273
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.12batch/s]
          

Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713433624623896
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.25batch/s]
                                                                    INFO: Validation Dice Loss: 0.37135128333018375
Epoch 6/8: 100%|██████████| 5865/5865 [02:11<00:00, 44.51img/s, loss (batch)=0.696]
INFO: Checkpoint 6 saved !
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 24.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.37137282615179545
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.19batch/s]
                                                                    INFO: Validation Dice Loss: 0.37138476967811584
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 26.31batch/s]
                                                                    INFO: Validat

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.37batch/s]
                                                                    INFO: Validation Dice Loss: 0.37138751738674036
Validation round: 100%|██████████| 91/91 [00:04<00:00, 24.72batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713679546183282
Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.60batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713565141290099
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713393384938712
Epoch 7/8: 100%|██████████| 5865/5865 [02:11<00:00, 44.57img/s, loss (batch)=0.695]
INFO: Checkpoint 7 saved !
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 26.23batch/s]
                                                                    INFO: Validatio

Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 25.97batch/s]
                                                                    INFO: Validation Dice Loss: 0.37135501779042757
Validation round: 100%|██████████| 91/91 [00:04<00:00, 23.61batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713607974759825
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.99batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713555899295178
Validation round:  99%|█████████▉| 90/91 [00:04<00:00, 24.29batch/s]
                                                                    INFO: Validation Dice Loss: 0.37133056547615556
Validation round: 100%|██████████| 91/91 [00:04<00:00, 25.97batch/s]
                                                                    INFO: Validation Dice Loss: 0.3713051437676608
Validation round: 100%|██████████| 91/91 [00:04<00:00, 26.45batch/s]
         

In [12]:
net8 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=256, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net8.to(device=device)
train_scores8, val_scores8, train_var_8, val_var_8 = train_net(net=net8,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.83batch/s]
                                                                    INFO: Validation Dice Loss: 0.4007226520187252
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.40batch/s]
                                                                    INFO: Validation Dice Loss: 0.38232069683598946
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.59batch/s]
                                                                    INFO: Validation Dice Loss: 0.42474764257996944
Validation round:  98%|█████████▊| 89/

Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.18batch/s]
                                                                    INFO: Validation Dice Loss: 0.44876017413296543
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.48batch/s]
                                                                    INFO: Validation Dice Loss: 0.4428297736487546
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.58batch/s]
                                                                    INFO: Validation Dice Loss: 0.43245874852924554
Epoch 1/8: 100%|██████████| 5865/5865 [02:35<00:00, 37.74img/s, loss (batch)=0.835]
INFO: Checkpoint 1 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.92batch/s]
                                                                    INFO: Validation Dice Loss: 0.4388227318669414
Validation round:  92%|█████████▏| 84/91 [00:05<00:00, 19.39batch/s]


Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.56batch/s]
                                                                    INFO: Validation Dice Loss: 0.442921498974601
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.44661774379866465
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.33batch/s]
                                                                    INFO: Validation Dice Loss: 0.44354443104712515
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.51batch/s]
                                                                    INFO: Validation Dice Loss: 0.44463173013467056
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.44477541204337234
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.08batch/s]
        

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.4456879090476822
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.44539724666993696
Epoch 2/8: 100%|██████████| 5865/5865 [02:39<00:00, 36.79img/s, loss (batch)=0.85] 
INFO: Checkpoint 2 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.27batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454477072417081
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454661394868578
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.41batch/s]
                                                                    INFO: Validatio

Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.17batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454529874927395
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.51batch/s]
                                                                    INFO: Validation Dice Loss: 0.44544582019795426
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.77batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454801076061123
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.46batch/s]
                                                                    INFO: Validation Dice Loss: 0.44545940776447673
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.28batch/s]
                                                                    INFO: Validation Dice Loss: 0.44541935999314863


...

Validation round: 100%|██████████| 91/91 [00:06<00:00, 16.19batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454168002684038
Validation round: 100%|██████████| 91/91 [00:06<00:00, 14.85batch/s]
                                                                    INFO: Validation Dice Loss: 0.4453709328567589
Validation round: 100%|██████████| 91/91 [00:06<00:00, 15.11batch/s]
                                                                    INFO: Validation Dice Loss: 0.4453757365981301
Epoch 6/8: 100%|██████████| 5865/5865 [03:04<00:00, 31.78img/s, loss (batch)=0.853]
INFO: Checkpoint 6 saved !
Validation round: 100%|██████████| 91/91 [00:07<00:00, 14.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454161803145985
Validation round:  54%|█████▍    | 49/91 [00:03<00:02, 14.85batch/s]


Validation round: 100%|██████████| 91/91 [00:06<00:00, 15.92batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454017582830492
Validation round: 100%|██████████| 91/91 [00:07<00:00, 13.79batch/s]
                                                                    INFO: Validation Dice Loss: 0.4453887932903164
Validation round: 100%|██████████| 91/91 [00:06<00:00, 15.01batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454006097473941
Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.95batch/s]
                                                                    INFO: Validation Dice Loss: 0.4453238009751498
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.87batch/s]
                                                                    INFO: Validation Dice Loss: 0.4452967165590643
Validation round:  29%|██▊       | 26/91 [00:01<00:04, 15.46batch/s]


Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.23batch/s]
                                                                    INFO: Validation Dice Loss: 0.4453111042033185
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.49batch/s]
                                                                    INFO: Validation Dice Loss: 0.44535156298469714
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.70batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454304327021588
Epoch 7/8: 100%|██████████| 5865/5865 [02:59<00:00, 32.66img/s, loss (batch)=0.822]
INFO: Checkpoint 7 saved !
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.94batch/s]
                                                                    INFO: Validation Dice Loss: 0.4455172504697527
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.27batch/s]
                                                                    INFO: Validatio

Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.08batch/s]
                                                                    INFO: Validation Dice Loss: 0.44549821137071965
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454414209166726
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.63batch/s]
                                                                    INFO: Validation Dice Loss: 0.445393089737211
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454170612843482
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.08batch/s]
                                                                    INFO: Validation Dice Loss: 0.44540611466208657
Validation round:  43%|████▎     | 39/91 [00:02<00:02, 17.76batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.25batch/s]
                                                                    INFO: Validation Dice Loss: 0.4454291079725538
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.44546190067961977
Epoch 8/8: 100%|██████████| 5865/5865 [02:39<00:00, 36.83img/s, loss (batch)=0.82] 
INFO: Checkpoint 8 saved !


In [1]:
print("configuring combined plots")
domain = len(train_scores1)
x_values = [i+1 for i in range(domain)]
             
a1 = plt.subplot(1,2,1)
a1.set_ylim([0, 0.5])
plt.title("Complex SLN 64 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores1, train_var_1)]
ziptraindown = [a - b for a, b in zip(train_scores1, train_var_1)]
zipvalup = [a + b for a, b in zip(val_scores1, val_var_1)]
zipvaldown = [a - b for a, b in zip(val_scores1, val_var_1)]

plt.plot(x_values, train_scores1, color="blue", label="train")
a1.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores1, color="orange", label="val")
a1.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a2 = plt.subplot(1,2,2)
a2.set_ylim([0, 0.5])
plt.title("Complex SLN 128 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores2, train_var_2)]
ziptraindown = [a - b for a, b in zip(train_scores2, train_var_2)]
zipvalup = [a + b for a, b in zip(val_scores2, val_var_2)]
zipvaldown = [a - b for a, b in zip(val_scores2, val_var_2)]

plt.plot(x_values, train_scores2, color="blue", label="train")
a2.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores2, color="orange", label="val")
a2.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

a3 = plt.subplot(1,2,1)
a3.set_ylim([0, 0.5])
plt.title("Complex SLN 200 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores3, train_var_3)]
ziptraindown = [a - b for a, b in zip(train_scores3, train_var_3)]
zipvalup = [a + b for a, b in zip(val_scores3, val_var_3)]
zipvaldown = [a - b for a, b in zip(val_scores3, val_var_3)]

plt.plot(x_values, train_scores3, color="blue", label="train")
a3.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores3, color="orange", label="val")
a3.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a4 = plt.subplot(1,2,2)
a4.set_ylim([0, 0.5])
plt.title("Complex SLN 256 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores4, train_var_4)]
ziptraindown = [a - b for a, b in zip(train_scores4, train_var_4)]
zipvalup = [a + b for a, b in zip(val_scores4, val_var_4)]
zipvaldown = [a - b for a, b in zip(val_scores4, val_var_4)]

plt.plot(x_values, train_scores4, color="blue", label="train")
a4.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores4, color="orange", label="val")
a4.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

a5 = plt.subplot(1,2,1)
a5.set_ylim([0, 0.5])
plt.title("AE SLN 64 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores5, train_var_5)]
ziptraindown = [a - b for a, b in zip(train_scores5, train_var_5)]
zipvalup = [a + b for a, b in zip(val_scores5, val_var_5)]
zipvaldown = [a - b for a, b in zip(val_scores5, val_var_5)]

plt.plot(x_values, train_scores5, color="blue", label="train")
a5.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores5, color="orange", label="val")
a5.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a6 = plt.subplot(1,2,2)
a6.set_ylim([0, 0.5])
plt.title("AE SLN 128 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores6, train_var_6)]
ziptraindown = [a - b for a, b in zip(train_scores6, train_var_6)]
zipvalup = [a + b for a, b in zip(val_scores6, val_var_6)]
zipvaldown = [a - b for a, b in zip(val_scores6, val_var_6)]

plt.plot(x_values, train_scores6, color="blue", label="train")
a6.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores6, color="orange", label="val")
a6.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

a7 = plt.subplot(1,2,1)
a7.set_ylim([0, 0.5])
plt.title("AE SLN 200 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores7, train_var_7)]
ziptraindown = [a - b for a, b in zip(train_scores7, train_var_7)]
zipvalup = [a + b for a, b in zip(val_scores7, val_var_7)]
zipvaldown = [a - b for a, b in zip(val_scores7, val_var_7)]

plt.plot(x_values, train_scores7, color="blue", label="train")
a7.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores7, color="orange", label="val")
a7.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a8 = plt.subplot(1,2,2)
a8.set_ylim([0, 0.5])
plt.title("AE SLN 256 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores8, train_var_8)]
ziptraindown = [a - b for a, b in zip(train_scores8, train_var_8)]
zipvalup = [a + b for a, b in zip(val_scores8, val_var_8)]
zipvaldown = [a - b for a, b in zip(val_scores8, val_var_8)]

plt.plot(x_values, train_scores8, color="blue", label="train")
a8.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores8, color="orange", label="val")
a8.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

plt.close()

configuring combined plots


NameError: name 'train_scores1' is not defined

In [14]:
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_64.npy",net1.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_128.npy",net2.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_200.npy",net3.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_256.npy",net4.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/AE_64.npy",net5.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/AE_128.npy",net6.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/AE_200.npy",net7.W.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/superblocks/AE_256.npy",net8.W.cpu().data)